In [1]:
import os
from models.midi_transformer import MIDITransformer
from models.helpers.blocks import TransformerXLEncoderStack, TransformerXLDecoderStack
from models.helpers.utils import positional_encoding
import tensorflow as tf

In [2]:
BASE_DIR = "/home/richhiey/Desktop/workspace/projects/virtual_musicians"
DATA_DIR = os.path.join(BASE_DIR, 'data', 'POP909-Dataset')
MODEL_SAVE_PATH = os.path.join(BASE_DIR, 'checkpoints', 'model1')
MODEL_CONFIG_PATH = os.path.join(BASE_DIR, 'model-store', 'models', 'configs', 'default.json')

In [3]:
midi_transformer = MIDITransformer(MODEL_CONFIG_PATH, MODEL_SAVE_PATH)

inputx = tf.random.uniform(shape=(10,256), maxval=128, dtype=tf.int32)
outputx = tf.random.uniform(shape=(10,256), maxval=128, dtype=tf.int32)
pe = tf.random.uniform(shape=(512,128), dtype=tf.float32)
pe = tf.tile(tf.expand_dims(pe, axis=0), [10, 1, 1])
memory_length = tf.constant(256)
look_ahead_mask = tf.expand_dims(tf.tile(tf.sequence_mask([200], 256), [10,1]), axis=0)

midi_transformer(inputx, outputx, pe, look_ahead_mask, memory_length)

Initializing Encoder from scratch.
Initializing Decoder from scratch.
Encoder output - 
tf.Tensor([ 10 256 128], shape=(3,), dtype=int32)
Decoder output - 
tf.Tensor([ 10 256 128], shape=(3,), dtype=int32)


(<tf.Tensor: shape=(10, 256, 128), dtype=float32, numpy=
 array([[[-3.60262632e-01, -9.10086989e-01,  2.15590525e+00, ...,
           2.64191836e-01,  2.75574774e-01,  1.93990529e+00],
         [ 8.64221632e-01, -2.06788555e-01,  3.77180874e-01, ...,
           5.11551142e-01, -1.46557772e+00,  1.44500470e+00],
         [-6.76668435e-02,  7.56184816e-01, -8.96295369e-01, ...,
          -5.18232942e-01, -1.04306984e+00,  1.09082234e+00],
         ...,
         [-3.02245617e-01, -4.54290688e-01,  4.00730550e-01, ...,
           8.90224695e-01, -1.21039069e+00,  1.63135815e+00],
         [-9.00536537e-01, -1.16570282e+00,  7.68307805e-01, ...,
          -7.81343460e-01, -1.18737721e+00,  2.31590796e+00],
         [ 7.58257508e-01, -4.83377457e-01,  9.17776227e-02, ...,
           8.30856502e-01, -9.43109691e-01,  1.20872855e+00]],
 
        [[ 6.52724028e-01, -4.68820147e-02, -4.37540948e-01, ...,
          -1.17569494e+00, -1.14730692e+00,  6.46405458e-01],
         [ 9.89895642e-01, -1.